In [3]:
from pyscf import gto
# mol = gto.M(atom='O	0.0000000	0.0000000	0.1172700; H	0.0000000	0.7569520	-0.4690790; H	0.0000000	-0.7569520	-0.4690790', basis='ccpvdz')
mol = gto.M(atom='F 0 0 0; F 0 0 0.9', basis='ccpvdz')

In [4]:
# static polarizability module

from pyscf import gto, dft, grad, scf
import numpy as np
from pyscf.scf.hf import RHF
from pyscf.dft.rks import RKS
from pyscf.scf.hf import dip_moment

# Hartree-Fork SCF Method

def hcore_caused_by_elec_field(mol, e_field=np.array([0,0,0])):
    h1 = mol.intor('int1e_r')
    return np.einsum('xij,x->ij', h1, e_field)

class ModifiedRHF(RHF):
    def __init__(self, mol, e_field=np.zeros(3)):
        RHF.__init__(self, mol)
        self.e_field = e_field
        
    def get_hcore(self, mol=None):
        if mol is None: mol = self.mol
        return RHF.get_hcore(self, mol) + hcore_caused_by_elec_field(mol, self.e_field)

def get_dipole_with_field_RHF(mol, e_field=np.zeros(3)):
    mf = ModifiedRHF(mol, e_field=e_field)
    mf.conv_tol_grad = 1e-8
    mf.max_cycle = 100
    mf.kernel()
    dm = mf.make_rdm1()
    return dip_moment(mol, dm, unit='AU')

def get_dipole_at_grid_RHF(mol, direction, h):
    # preparation for central difference
    if direction == 'x':
        elec_grid = np.array([[i * h, 0, 0] for i in range(-2,3)])
    elif direction == 'y':
        elec_grid = np.array([[0, i * h, 0] for i in range(-2,3)])
    elif direction == 'z':
        elec_grid = np.array([[0, 0, i * h] for i in range(-2,3)])
    
    dipole_at_grid = []
    for e_field in elec_grid:
        dipole_at_grid.append(get_dipole_with_field_RHF(mol, e_field))
    
    return np.array(dipole_at_grid)

def get_polarizability_RHF(mol):
    # 4th order central difference
    h = 0.001
    central_difference_coeff = np.array([1/12, -2/3, 0, 2/3, -1/12])/h
    
    # calculate polarizability
    polarizability_caused_by_ex = np.dot(get_dipole_at_grid_RHF(mol, 'x', h).T, central_difference_coeff)
    polarizability_caused_by_ey = np.dot(get_dipole_at_grid_RHF(mol, 'y', h).T, central_difference_coeff)
    polarizability_caused_by_ez = np.dot(get_dipole_at_grid_RHF(mol, 'z', h).T, central_difference_coeff)
    
    # keep the column vector format
    return np.array([polarizability_caused_by_ex, polarizability_caused_by_ey, polarizability_caused_by_ez]).T

def get_beta_RHF(mol):
    alpha = get_polarizability_RHF(mol)
    a = np.trace(alpha)/3
    beta = alpha - a*np.identity(3)
    return beta

# DFT Method

class ModifiedRKS(RKS):
    def __init__(self, mol, xc='LDA,VWN', e_field=np.zeros(3)):
        RKS.__init__(self, mol, xc=xc)
        self.e_field = e_field
        
    def get_hcore(self, mol=None):
        if mol is None: mol = self.mol
        return RKS.get_hcore(self, mol) + hcore_caused_by_elec_field(mol, self.e_field)

def get_dipole_with_field_DFT(mol, e_field=np.zeros(3)):
    mf = ModifiedRKS(mol, e_field=e_field, xc='b3lyp')
    mf.conv_tol_grad = 1e-8
    mf.max_cycle = 100
    mf.kernel()
    dm = mf.make_rdm1()
    return dip_moment(mol, dm, unit='A.U.')

def get_dipole_at_grid_DFT(mol, direction, h):
    if direction == 'x':
        elec_grid = np.array([[i * h, 0, 0] for i in range(-2,3)])
    elif direction == 'y':
        elec_grid = np.array([[0, i * h, 0] for i in range(-2,3)])
    elif direction == 'z':
        elec_grid = np.array([[0, 0, i * h] for i in range(-2,3)])
    
    dipole_at_grid = []
    for e_field in elec_grid:
        dipole_at_grid.append(get_dipole_with_field_DFT(mol, e_field))
    
    return np.array(dipole_at_grid)

def get_polarizability_DFT(mol):
    # 4th order central difference
    h = 0.001
    central_difference_coeff = np.array([1/12, -2/3, 0, 2/3, -1/12])/h
    
    # calculate polarizability
    polarizability_caused_by_ex = np.dot(get_dipole_at_grid_DFT(mol, 'x', h).T, central_difference_coeff)
    polarizability_caused_by_ey = np.dot(get_dipole_at_grid_DFT(mol, 'y', h).T, central_difference_coeff)
    polarizability_caused_by_ez = np.dot(get_dipole_at_grid_DFT(mol, 'z', h).T, central_difference_coeff)
    
    # keep the column vector format
    return np.array([polarizability_caused_by_ex, polarizability_caused_by_ey, polarizability_caused_by_ez]).T

def get_beta_DFT(mol):
    alpha = get_polarizability_DFT(mol)
    a = np.trace(alpha)/3
    beta = alpha - a*np.identity(3)
    return beta

In [5]:
polar = get_polarizability_RHF(mol)

converged SCF energy = -198.152862181388
Dipole moment(X, Y, Z, A.U.): -0.00315,  0.00000,  0.00000
converged SCF energy = -198.152859819304
Dipole moment(X, Y, Z, A.U.): -0.00157, -0.00000,  0.00000


<class '__main__.ModifiedRHF'> does not have attributes  e_field


converged SCF energy = -198.152859031942
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00000, -0.00000
converged SCF energy = -198.152859819304
Dipole moment(X, Y, Z, A.U.):  0.00157,  0.00000,  0.00000
converged SCF energy = -198.152862181388
Dipole moment(X, Y, Z, A.U.):  0.00315, -0.00000,  0.00000
converged SCF energy = -198.152862181388
Dipole moment(X, Y, Z, A.U.): -0.00000, -0.00315,  0.00000
converged SCF energy = -198.152859819304
Dipole moment(X, Y, Z, A.U.): -0.00000, -0.00157,  0.00000
converged SCF energy = -198.152859031942
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00000, -0.00000
converged SCF energy = -198.152859819304
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00157,  0.00000
converged SCF energy = -198.152862181388
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00315,  0.00000
converged SCF energy = -198.18347869851
Dipole moment(X, Y, Z, A.U.):  0.00000,  0.00000, -0.00610
converged SCF energy = -198.168167339388
Dipole moment(X, Y, Z, A.U.): -0.00000, -0.00000, -0.00305
c

In [4]:
polar

array([[ 3.04024876e+00, -6.51490288e-14, -2.33858703e-13],
       [-1.70393956e-28,  6.91121058e+00,  1.38777878e-13],
       [-2.74445380e-14,  7.83367477e-12,  5.08919005e+00]])

In [6]:
# normal modes

from pyscf.hessian import thermo
from pyscf.hessian import rhf

# mf = scf.RHF(mol)
# mf.kernel()

# hessian_calculator = rhf.Hessian(mf)
# hess = hessian_calculator.kernel()

# Perform harmonic analysis
# freq = thermo.harmonic_analysis(mol, hess)

# Output normal modes
# normal_modes = freq['norm_mode']
# wave_number = freq['freq_wavenumber']

# print(normal_modes)

In [7]:
from pyscf.hessian import thermo
from pyscf.hessian import rhf

def displacement_each_mode(mol, mode, magnitude):
    new_mol = mol.copy()
    new_atom_coords = mol.atom_coords(unit='Angstrom') + mode * magnitude
    elements = mol.elements
    
    atom_lines = []
    for i in range(len(elements)):
        each_atom = elements[i] + ' ' + str(new_atom_coords[i][0]) + ' ' + str(new_atom_coords[i][1]) + ' ' + str(new_atom_coords[i][2])
        atom_lines.append(each_atom)
        
    new_geometry = '; '.join(atom_lines)
    new_mol.atom = new_geometry
    new_mol.build()
    return new_mol

# print(mol.atom)
# new_mol = displacement_each_mode(mol, normal_modes[0],1)
# print(new_mol.atom)

In [8]:
def polarizability_derivative_each_mode_RHF(mol, mode):
    h = 0.001
    # 4th order central difference
    displacement_grid = np.array([i * h for i in range(-2,3)])
    central_difference_coeff = np.array([1/12, -2/3, 0, 2/3, -1/12])/h
    
    polarizability_at_grid = []
    for displacement in displacement_grid:
        new_mol = displacement_each_mode(mol, mode, displacement)
        polarizability_at_grid.append(get_polarizability_RHF(new_mol))
        
    polarizability_at_grid = np.array(polarizability_at_grid)
    polarizability_derivative = np.einsum('xij,x->ij', polarizability_at_grid, central_difference_coeff)
    
    return polarizability_derivative

# polar_derivative = polarizability_derivative_each_mode_RHF(mol, normal_modes[0])
# print(polar_derivative)

In [9]:
def activity_each_mode(polar_derivative):
    
    # space average invariants
    a_2 = (np.trace(polar_derivative)/3)**2
    gamma_2 = 0.5*(polar_derivative[0][0] - polar_derivative[1][1])**2 + 0.5*(polar_derivative[1][1] - polar_derivative[2][2])**2 + 0.5*(polar_derivative[2][2] - polar_derivative[0][0])**2 + 3*polar_derivative[0][1]**2 + 3*polar_derivative[1][2]**2 + 3*polar_derivative[2][0]**2
    
    # calculate raman activity
    activity_of_mode = (45*a_2+7*gamma_2)/45
    return activity_of_mode

# print(activity_each_mode(polar_derivative))

In [10]:
def get_raman_activities_RHF(mol): 
    # calculate normal modes
    mf = scf.RHF(mol)
    mf.kernel()
    hessian_calculator = rhf.Hessian(mf)
    hess = hessian_calculator.kernel()
    
    # Perform harmonic analysis
    freq = thermo.harmonic_analysis(mol, hess)
    normal_modes = freq['norm_mode']
    
    # calculate raman activity
    raman_activities = []
    for mode in normal_modes:
        polar_derivative = polarizability_derivative_each_mode_RHF(mol, mode)
        raman_activities.append(activity_each_mode(polar_derivative))
        
    return np.array(raman_activities)

result = get_raman_activities_RHF(mol)
print(result)

converged SCF energy = -198.152859031942
converged SCF energy = -198.150240994645
Dipole moment(X, Y, Z, A.U.): -0.00315, -0.00000,  0.00000
converged SCF energy = -198.150238632449
Dipole moment(X, Y, Z, A.U.): -0.00157,  0.00000, -0.00000
converged SCF energy = -198.15023784505
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00000, -0.00000
converged SCF energy = -198.150238632449
Dipole moment(X, Y, Z, A.U.):  0.00157, -0.00000, -0.00000
converged SCF energy = -198.150240994645
Dipole moment(X, Y, Z, A.U.):  0.00315,  0.00000,  0.00000
converged SCF energy = -198.150240994645
Dipole moment(X, Y, Z, A.U.):  0.00000, -0.00315,  0.00000
converged SCF energy = -198.150238632449
Dipole moment(X, Y, Z, A.U.): -0.00000, -0.00157,  0.00000
converged SCF energy = -198.15023784505
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00000, -0.00000
converged SCF energy = -198.150238632449
Dipole moment(X, Y, Z, A.U.): -0.00000,  0.00157,  0.00000
converged SCF energy = -198.150240994645
Dipole moment(X, Y, 

gaussian result for F2 is 1.2099